In [1]:
#import procedure
import pandas as pd
import numpy as np
import shutil

file = r'D:\LocalData\kucensj\Desktop\SV 2016 data.csv' #use r because of special characters
shutil.copyfile(file, 'test.csv')

na_values = [' ', 'N/A'] #define which text to convert to NaN
#default case is with headers; use header=None for no headers
    #added encoding='latin-1' due to open error; non-ascii characters???
df = pd.read_csv('test.csv', sep=',', na_values=na_values, encoding='latin-1') 
df = df.reindex(np.random.permutation(df.index)) #put rows in random order, then reset index
#print(df.head()) #prints first 5 rows by default
df = df.reset_index()
df = df.drop(["index"],axis=1) #delete column
df_row,df_col = df.shape #determine size of data

D:\WinPython-64bit-3.4.4.1\python-3.4.4.amd64\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (15,472,481) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
#Create dataframe with variable information
df_var = pd.DataFrame(df.axes[1])
df_var.columns = ['Variable']
dt = [df.dtypes[i] for i in range(0,df_col)] 
df_var['VarType'] = dt #add new column
df_var.index = df_var['Variable']
#df_var[:10]

#Get basic statistics
df_stats = df.describe().T  #transpose results to match format of df_var
df_stats = df_stats.reset_index()

df_var = pd.merge(df_var, df_stats, how='left', left_on='Variable', right_on='index')

#df_stats.index = df_stats['index']  #Join based on variable name
#df_var = pd.concat([df_var,df_stats], axis=1)  
  #Note, pd.merge is more powerful than pd.concat when using multiple criteria

df_var = df_var.drop(['index'], axis=1)  #remove the duplicate index
df_var.to_csv("df_vars.csv")

In [6]:
#Create correlation & covariance matrix
s = df_var[df_var['VarType']=='float64'][['Variable']] #List of numeric variables
s = s.values.tolist() #Convert data frame to list of lists
filter = [i[0] for i in s] #Collapse sublists to first item
df_num = df[filter] #subset dataframe with numeric variables
df_num_row,df_num_col = df_num.shape
del(s, filter)

df_corr = df_num.corr()
df_cov = df_num.cov()

#Create tuple to return correlation values
corr_vec = [(df_corr.index[x], df_corr.columns[y], df_corr.iloc[x,y], df_cov.iloc[x,y]) for x in range(0,df_num_col-1) for y in range(0,df_num_col-1) if y > x]

#Convert tuple a dataframe and add labels
df_corr_vec = pd.DataFrame(corr_vec, columns=['Variable1', 'Variable2', 'Correlation', 'Covariance'])
filter1 = df_corr_vec['Correlation']<=-.6
filter2 = df_corr_vec['Correlation']>=.6
df_corr_vec = df_corr_vec[filter1 | filter2]
del(filter1, filter2)

df_corr_vec = df_corr_vec.sort_values(by='Correlation')
df_corr_vec.to_csv("df_corr_vec.csv")

In [3]:
#Now manually use correlation file and statistics file to determine which variables to eliminate
#Also determine which categorical variables to include in the analysis

#Import list of variables to exclude from next analyses
df_excl = pd.read_csv('df_excl.csv', sep=',', na_values=na_values, encoding='latin-1') 
s = df_var[df_var['VarType']=='float64'][['Variable']].values.tolist()
var_list = [i[0] for i in s] #Collapse sublists to first item
s = df_excl.values.tolist() #Convert data frame to list of lists
not_in = [i[0] for i in s] #Collapse sublists to first item

#Add categorical variables
df_incl = pd.read_csv('df_incl.csv', sep=',', na_values=na_values, encoding='latin-1') 
s = df_incl.values.tolist() #Convert data frame to list of lists
s = [i[0] for i in s] #Collapse sublists to first item
df_cat = pd.DataFrame(s) #save categorical variables
s.extend(var_list)
var_list = s
var_list = [i for i in var_list if i not in not_in]

df_clean = df[var_list] #remove excluded variables from analysis
del(s, df_excl, not_in, var_list)

df_clean.head()

,0_New Model Purchased (Alpha Order),0_New Model Purchased - Brand,0_New Model Segment,0_Segmentation 1,0_Nissan Segment,0_Fuel Type,D_Gender,D_Lifestage,D_Lifestage Variable,D_Lifestage Plus Affluence,...,S_Future Trade-In Or Resale Value,S_Warranty Coverage,S_Technical Innovations,S_Level of Standard Equipment,S_Fuel Economy/Mileage,S_Affordable to Buy,S_Overall Image of the Vehicle,S_OVERALL Experience with Selling Dealership,S_Convenience of Dealer Location,S_Overall Experience with Service Dept.
0,Audi A4/S4 Sdn,Audi,Near Luxury,Luxury Mid-Size Car,Compact Premium Car,Flexible (i.e. ethanol),Male,Married Without Children,Post-Family (No Children/Age Res 45+),NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Toyota Tacoma,Toyota,Compact Pickup,Mid-Size Pickup,Midsize Pickup,Gas,Male,NaN,NaN,NaN,...,5,4,4,4,2,4,3,4,3,NaN
2,Honda Civic Sdn,Honda,Small Car,Compact Car,Compact Car,Gas,Female,Single Without Children,Post-Family (No Children/Age Res 45+),Post-Family - Income $80K or Less,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Dodge Journey,Dodge,Mid-Size Crossover Utility,Mid-size Crossover,Midsize SUV,NaN,Female,Married With Children,Younger-Family (Age Oldest Child 1-12),Younger-Family - Income $80K or Less,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Honda Civic Sdn,Honda,Small Car,Compact Car,Compact Car,NaN,Female,Other Without Children,Post-Family (No Children/Age Res 45+),Post-Family - Income $80K or Less,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
#Significance Testing Function
from scipy.stats import t
from math import sqrt

def sig_test(m1, n1, m2, n2, max_val=10, stdev1=0, stdev2=0): 
    #calculate t-test depending on input data
    #values should be in decimal form, not PPH
    #Results >1 are significant with 90% confidence
    #stdev is optional; not required for percentage testing
    
    Result = []
    if n1 == 0 or n2 == 0:
        Result = 0
    
    elif max_val > 1: #Calculate standard t-test
        #stdev defaulted to 1 if every customer has same rating (small sample size condition)
        if stdev1 == 0:
            stdev1 = 1
        if stdev2 == 0:
            stdev2 = 1
            
        Result = abs(m1 - m2) / sqrt(stdev1**2 / n1 + stdev2**2 / n2) / t.isf(.05, min(n1, n2) - 1)
    
    else: #Calculate based on Wilson method
        Result = max(0,(2 * max(m1 , m2) - (m1 * n1 + 1.645**2 / 2) / (n1 + 1.645**2) - (m2 * n2 + 1.645**2 / 2) / (n2 + 1.645**2)) / 1.645 / sqrt((m1 * (1 - m1) * n1 + 1.645**2 / 4) / ((n1 + 1.645**2)**2) + (m2 * (1 - m2) * n2 + 1.645**2 / 4) / (n2 + 1.645**2)**2))
    
    return (Result)

In [8]:
#Slice dataset and compare results (Version 1)
segments = df_clean['0_Nissan Segment'].unique()

y=len(df_cat)

df_stats = df_clean.describe().T
row,col = df_stats.shape
subset = ['Average' for i in range(0,row)]
df_stats['Subset'] = subset
df_comp = pd.DataFrame(columns=['Subset', 'Subset', 'mean', 'mean', 'count', 'count', 'std', 'std', 'Significance'])
df_comp.reset_index()

#Slice by segment
for segment in segments:
    df_sub = df_clean[df_clean['0_Nissan Segment']==segment].describe().T
    row,col = df_sub.shape
    subset = [segment for i in range(0,row)]
    df_sub['Subset'] = subset
    df_stats = pd.concat([df_stats,df_sub])
    
    #Compare segments vs average
    df_ttest = pd.concat([df_stats[df_stats['Subset']=='Average'],df_stats[df_stats['Subset']==segment]], axis = 1)
    
    sig = []
    #Columns hard coded due to duplicate column names; use print to confirm correct columns selected
    #print([df_ttest.columns[1], df_ttest.columns[0], df_ttest.columns[2], df_ttest.columns[7], df_ttest.columns[10], df_ttest.columns[9], df_ttest.columns[11]])
    for i in range(0,row):
        sig.append(sig_test(m1 = df_ttest.iloc[i,1], n1 = df_ttest.iloc[i,0], stdev1 = df_ttest.iloc[i,2], max_val = df_ttest.iloc[i,7], m2 = df_ttest.iloc[i,10], n2 = df_ttest.iloc[i,9], stdev2 = df_ttest.iloc[i,11]))
    
    #Multiple columns automatically included if they have the same name
    df_ttest['Significance'] = sig
    df_ttest = df_ttest[df_ttest['Significance']>=0.95][[df_ttest.columns[17], df_ttest.columns[10], df_ttest.columns[9], df_ttest.columns[11], df_ttest.columns[18]]]
    df_comp = pd.concat([df_comp, df_ttest])

#Correct variables with same name
df_comp.columns = ['Avg Subset', 'Subset', 'Avg_mean', 'Subset_mean', 'Avg_count', 'Subset_count', 'Avg_stdev', 'Subset_stdev', 'Significance']
df_comp = df_comp.drop(['Avg Subset'], axis=1)

df_stats.to_csv("df_stats.csv")
df_comp.to_csv("df_comp.csv")

In [ ]:
#Slice dataset and compare results (Version 2 -- Runs Slow)
slice = (('D_Occupation', 'occupation: '), ('D_Customer Region Classification #1', 'region: '), 
         ('D_State', 'state: '), ('D_DMA', 'DMA: '), ('0_New Model Purchased (Alpha Order)', 'model: '),
         ('D_Small Town Or Rural City', 'rural: '), ('Prob_Nature Of Problem On New Vehicle', 'problem: '),
         ('Shop_Seriously Consider Other Models ?', 'cross shop: '), ('Shop_Segment Most Seriously Considered', 'shop segment: '),
         ('Shop_Segment (Disposed)', 'dispose segment: '))

Filter1 = df_clean['0_Nissan Segment']=='Midsize Car'

df_stats = df_clean[Filter1].describe().T
row,col = df_stats.shape
subset = ['Midsize Car Average' for i in range(0,row)]
df_stats['Subset'] = subset
df_comp = pd.DataFrame(columns=['Subset', 'Subset', 'mean', 'mean', 'count', 'count', 'std', 'std', 'Significance'])
df_comp.reset_index()

for i in range(0, len(slice)):
    variable, label = slice[i]
    options = df_clean[variable].unique()
    
    for option in options:
        Filter2 = df_clean[variable]==option
        df_sub = df_clean[Filter1 & Filter2].describe().T
        row,col = df_sub.shape
        subset = [label + str(option) for i in range(0,row)]
        df_sub['Subset'] = subset
        df_stats = pd.concat([df_stats,df_sub])
        df_ttest = pd.concat([df_stats[df_stats['Subset']=='Midsize Car Average'],df_stats[df_stats['Subset']==label + str(option)]], axis = 1)

        sig = []
        for i in range(0,row):
            sig.append(sig_test(m1 = df_ttest.iloc[i,1], n1 = df_ttest.iloc[i,0], stdev1 = df_ttest.iloc[i,2], max_val = df_ttest.iloc[i,7], m2 = df_ttest.iloc[i,10], n2 = df_ttest.iloc[i,9], stdev2 = df_ttest.iloc[i,11]))

        df_ttest['Significance'] = sig
        df_ttest = df_ttest[df_ttest['Significance']>=0.95][[df_ttest.columns[17], df_ttest.columns[10], df_ttest.columns[9], df_ttest.columns[11], df_ttest.columns[18]]]
        df_comp = pd.concat([df_comp, df_ttest])

#Correct variables with same name
df_comp.columns = ['Avg Subset', 'Subset', 'Avg_mean', 'Subset_mean', 'Avg_count', 'Subset_count', 'Avg_stdev', 'Subset_stdev', 'Significance']
df_comp = df_comp.drop(['Avg Subset'], axis=1)

df_stats.to_csv("df_stats.csv")
df_comp.to_csv("df_comp.csv")

In [5]:
#Slice dataset and compare results (Version 3)
slice = (('D_Gender', 'gender: '), ('D_Marital Status', 'marital status: '), ('D_Lifestage', 'lifestage: '),
         ('D_Lifestage Variable', 'lifestage_2: '), ('D_Lifestage Plus Affluence', 'lifestage_3: '),
         ('D_Generation', 'generation: '), ('D_Education', 'education: '), ('D_Ethnic Group', 'ethnicity: '),
         ('D_Occupation', 'occupation: '), ('D_Customer Region Classification #1', 'region: '), 
         ('D_State', 'state: '), ('D_DMA', 'DMA: '), ('0_New Model Purchased (Alpha Order)', 'model: '),
         ('D_Location Of Residence', 'residence: '), ('Prob_Nature Of Problem On New Vehicle', 'problem: '),
         ('Shop_Seriously Consider Other Models ?', 'cross shop: '), ('Shop_Segment (Disposed)', 'dispose segment: '),
         ('Shop_Segment Most Seriously Considered', 'shop segment: '))

Filter1 = df_clean['0_Nissan Segment']=='Midsize Car'
df_clean_f = df_clean[Filter1]

df_stats = df_clean_f.describe().T
row,col = df_stats.shape
subset = ['Midsize Car Average' for i in range(0,row)]
df_stats['Subset'] = subset
df_comp = pd.DataFrame(columns=['Subset', 'Subset', 'mean', 'mean', 'count', 'count', 'std', 'std', 'Significance'])
df_comp.reset_index()

for i in range(0, len(slice)):
    variable, label = slice[i]
    options = df_clean_f[variable].unique()
    print(slice[i])
    
    for option in options:
        Filter2 = df_clean_f[variable]==option
        df_sub = df_clean_f[Filter2].describe().T
        row,col = df_sub.shape
        subset = [label + str(option) for i in range(0,row)]
        df_sub['Subset'] = subset
        df_stats = pd.concat([df_stats,df_sub])
        df_ttest = pd.concat([df_stats[df_stats['Subset']=='Midsize Car Average'],df_stats[df_stats['Subset']==label + str(option)]], axis = 1)

        sig = []
        for i in range(0,row):
            sig.append(sig_test(m1 = df_ttest.iloc[i,1], n1 = df_ttest.iloc[i,0], stdev1 = df_ttest.iloc[i,2], max_val = df_ttest.iloc[i,7], m2 = df_ttest.iloc[i,10], n2 = df_ttest.iloc[i,9], stdev2 = df_ttest.iloc[i,11]))

        df_ttest['Significance'] = sig
        df_ttest = df_ttest[df_ttest['Significance']>=0.95][[df_ttest.columns[17], df_ttest.columns[10], df_ttest.columns[9], df_ttest.columns[11], df_ttest.columns[18]]]
        df_comp = pd.concat([df_comp, df_ttest])

#Correct variables with same name
df_comp.columns = ['Avg Subset', 'Subset', 'Avg_mean', 'Subset_mean', 'Avg_count', 'Subset_count', 'Avg_stdev', 'Subset_stdev', 'Significance']
df_comp = df_comp.drop(['Avg Subset'], axis=1)

df_stats.to_csv("df_stats.csv")
df_comp.to_csv("df_comp.csv")

('D_Gender', 'gender: ')
('D_Lifestage', 'lifestage: ')
('D_Lifestage Variable', 'lifestage_2: ')
('D_Lifestage Plus Affluence', 'lifestage_3: ')
('D_Education', 'education: ')
('D_Ethnic Group', 'ethnicity: ')
('D_Occupation', 'occupation: ')
('D_Customer Region Classification #1', 'region: ')
('D_State', 'state: ')
('D_DMA', 'DMA: ')
('0_New Model Purchased (Alpha Order)', 'model: ')
('D_Location Of Residence', 'residence: ')
('Prob_Nature Of Problem On New Vehicle', 'problem: ')
('Shop_Seriously Consider Other Models ?', 'cross shop: ')
('Shop_Segment (Disposed)', 'dispose segment: ')
('Shop_Segment Most Seriously Considered', 'shop segment: ')
